In [1]:
from tqdm.notebook import tqdm, trange

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import BaichuanTextEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.llms import OpenAI
from langchain_community.llms.chatglm3 import ChatGLM3
from langchain.chains.question_answering import load_qa_chain

from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/site-packages/awq/modules/linear/exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
/usr/local/lib/python3.10/site-packages/awq/modules/linear/exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"AutoAWQ could not load ExLlamaV2 kernels extension. Details: {ex}")
/usr/local/lib/python3.10/site-packages/awq/modules/linear/gemm.py:14: UserWarning: AutoAWQ could

In [4]:
# loader = UnstructuredPDFLoader("book/1.pdf")
loader = PyPDFLoader("book/1.pdf")
pages = loader.load_and_split()
print(pages[:1])  # Check the first page content

[Document(metadata={'source': 'book/1.pdf', 'page': 0}, page_content='第一回 桃园结义\n  \u3000\u3000东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与\n他的弟弟张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。\n\u3000\u3000没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向官府，声势非常浩大。汉灵帝得到各地报告，连忙\n下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领一精一兵，分路攻打张角兄弟的黄巾军。\n\u3000\u3000张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招\n募兵马。\n\u3000\u3000榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进\n城来看榜文。\n\u3000\u3000刘备看完了榜文，不觉感慨地长叹了一声。忽听身后有个人大声喝道∶“大丈夫不给国家出力，叹什么\n气？”\n\u3000\u3000刘备回头一看，这人身高八尺，豹子头，圆眼睛，满腮的胡须像钢丝一样竖着，声音像洪钟，样子十分威\n武。那人对刘备说他姓张名飞，字翼德，做着卖酒、屠宰猪羊的生意。他愿意拿出家产作本钱，与刘备共同干\n一番大事业。\n\u3000\u3000刘备、张飞两人谈得投机，便一起到村 口的一家酒 店饮酒叙话。\n\u3000\u3000这时，一推车大汉进店饮酒。刘备 留神一看，这人有 九尺高，胸前长须飘飘，脸色好像红枣一样，长一 双\n丹凤眼，两条卧蚕眉，相貌非常威武雄 伟。刘备连忙起身， 邀他过来同坐，并请问姓名。\n\u3000\u3000那人说∶“ 我姓关名羽，字云长，因乡里恶霸仗势欺人，我一怒杀了恶霸，逃到外乡避难已有五、六年\n了。”刘备、张飞听了 都很敬佩，也将自己的志愿告诉了关羽。关羽听了也非常高兴。\n\u3000\u3000酒后他 们一同来到张飞的 庄上，只见庄后有一座桃园，园中桃 花灿烂，景色很美。第二天，三人在园中 焚\n香礼拜，宣誓结为异姓兄弟，三人 按年岁认了兄弟，刘备做了大 哥，关羽第二，张飞最小，做了弟弟。\n\u3000\u3000三人 请来铁匠打造兵器。刘

In [11]:
os.environ.get("BAICHUAN_API_KEY")

In [13]:
# embeddings = OpenAIEmbeddings()
embeddings = BaichuanTextEmbeddings(api_key=baichuan_auth_token)

docsearch = Chroma.from_documents(pages, embeddings).as_retriever()
docsearch

ValidationError: 1 validation error for BaichuanTextEmbeddings
__root__
  Did not find baichuan_auth_token, please add an environment variable `BAICHUAN_AUTH_TOKEN` which contains it, or pass `baichuan_auth_token` as a named parameter. (type=value_error)

In [18]:
query = "桃源三结义都有谁？"
docs = docsearch.get_relevant_documents(query)
print(docs[:1])   # Ensure the document search returns relevant content

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'page': 0, 'source': 'book/1.pdf'}, page_content='第一回 桃园结义\n  \u3000\u3000东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与\n他的弟弟张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。\n\u3000\u3000没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向官府，声势非常浩大。汉灵帝得到各地报告，连忙\n下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领一精一兵，分路攻打张角兄弟的黄巾军。\n\u3000\u3000张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招\n募兵马。\n\u3000\u3000榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进\n城来看榜文。\n\u3000\u3000刘备看完了榜文，不觉感慨地长叹了一声。忽听身后有个人大声喝道∶“大丈夫不给国家出力，叹什么\n气？”\n\u3000\u3000刘备回头一看，这人身高八尺，豹子头，圆眼睛，满腮的胡须像钢丝一样竖着，声音像洪钟，样子十分威\n武。那人对刘备说他姓张名飞，字翼德，做着卖酒、屠宰猪羊的生意。他愿意拿出家产作本钱，与刘备共同干\n一番大事业。\n\u3000\u3000刘备、张飞两人谈得投机，便一起到村 口的一家酒 店饮酒叙话。\n\u3000\u3000这时，一推车大汉进店饮酒。刘备 留神一看，这人有 九尺高，胸前长须飘飘，脸色好像红枣一样，长一 双\n丹凤眼，两条卧蚕眉，相貌非常威武雄 伟。刘备连忙起身， 邀他过来同坐，并请问姓名。\n\u3000\u3000那人说∶“ 我姓关名羽，字云长，因乡里恶霸仗势欺人，我一怒杀了恶霸，逃到外乡避难已有五、六年\n了。”刘备、张飞听了 都很敬佩，也将自己的志愿告诉了关羽。关羽听了也非常高兴。\n\u3000\u3000酒后他 们一同来到张飞的 庄上，只见庄后有一座桃园，园中桃 花灿烂，景色很美。第二天，三人在园中 焚\n香礼拜，宣誓结为异姓兄弟，三人 按年岁认了兄弟，刘备做了大 哥，关羽第二，张飞最小，做了弟弟。\n\u3000\u3000三人 请来铁匠打造兵器。刘

In [15]:
chain = load_qa_chain(ChatGLM3(temperature=0), chain_type="stuff")
output = chain.run(input_documents=docs, question=query)
print(output)

ValueError: Missing some input keys: {'question'}